In [1]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
import os

from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from huggingface_hub import notebook_login
#from transformers import pipeline
#from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
import textwrap
import sys
import torch

from langchain.embeddings import OpenAIEmbeddings

In [2]:
import APIKEY
os.environ["OPENAI_API_KEY"] = APIKEY.API_KEY_SERVICE_OPENAI

## 1. Use unstructured to load pdf table
- We only get text here
- Alothough this unstructured can get table, but the quilty is not good.

In [3]:
from unstructured.partition.pdf import partition_pdf

DOC_FILE_NAME = "en-us--TRM_M463_M467_Series_EN_Rev1.01.pdf"
DOC_SAVE_NAME = "TRM_M463_M467"

#partition_pdf("layout-parser-paper-fast.pdf", url=None)

In [4]:
raw_pdf_elements = partition_pdf(filename=DOC_FILE_NAME,
                                 # Unstructured first finds embedded image blocks
                                 extract_images_in_pdf=False,
                                 # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
                                 # Titles are any sub-section of the document
                                 infer_table_structure=True,
                                 # Post processing to aggregate text once we have the title
                                 chunking_strategy="by_title",
                                 # Chunking params to aggregate text blocks
                                 # Attempt to create a new chunk 3800 chars
                                 # Attempt to keep chunks > 2000 chars
                                 max_characters=4000,
                                 new_after_n_chars=3800,
                                 combine_text_under_n_chars=2000)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 2779,
 "<class 'unstructured.documents.elements.Table'>": 3700,
 "<class 'unstructured.documents.elements.TableChunk'>": 98}

In [6]:
from pydantic import BaseModel
from typing import Any, Optional

class Element(BaseModel):
    type: str
    text: Any

# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))
    #elif "unstructured.documents.elements.Table" in str(type(element)):
    #    categorized_elements.append(Element(type="text", text=str(element)))

# Tables
#table_elements = [e for e in categorized_elements if e.type == "table"]
#print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

2779


In [7]:
print(type(text_elements[0]))
print(text_elements[0])

<class '__main__.Element'>
type='text' text='NuMicro® Family\n\nArm® Cortex® -M4-based Microcontroller\n\nM463/M467 Series Technical Reference Manual\n\nThe information described in this document is the exclusive intellectual property of Nuvoton Technology Corporation and shall not be reproduced without permission from Nuvoton.\n\nNuvoton is providing this document only for reference purposes of NuMicro® microcontroller and microprocessor based system design. Nuvoton assumes no responsibility for errors or omissions.\n\nAll data and specifications are subject to change without notice.\n\nFor additional information or questions, please contact: Nuvoton Technology Corporation.\n\nwww.nuvoton.com\n\nAug. 24, 2023\n\nPage 1 of 2380\n\nRev 1.01\n\nM 4 6 3 M 4 6 7 S E R E S T E C H N C A L R E F E R E N C E M A N U A L\n\n/\n\nI\n\nI\n\nTABLE OF CONTENTS\n\nM 4 6 3 M 4 6 7 S E R E S T E C H N C A L R E F E R E N C E M A N U A L\n\n/\n\nI\n\nI'


## 2. Use tabula to load pdf table

In [8]:
import tabula
from tabula import read_pdf
from tabulate import tabulate


# convert PDF into CSV
#tabula.convert_into("en-us--DS_NAU8822A_DataSheet_EN_Rev3.5.pdf", "NAU8822A_table.json", output_format="json", pages='3')
dfs = read_pdf(DOC_FILE_NAME, pages='36-2376', stream=True)
print(len(dfs))
 


Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'
Got stderr: 一月 31, 2024 2:05:29 下午 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored
一月 31, 2024 2:05:29 下午 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored
一月 31, 2024 2:05:30 下午 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored
一月 31, 2024 2:05:30 下午 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored
一月 31, 2024 2:05:34 下午 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored
一月 31, 2024 2:05:37 下午 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored
一月 31, 2024 2:05:51 下午 org.apache.fontbox.ttf.CmapSubtable processSubtype14


4004


In [9]:
# Transfer to str elements list
str_list = [df.to_csv(path_or_buf=None, index=False) for df in dfs]
#print(len(str_list))
#print(str_list[4])

table_elements = []
for str_df in str_list:    
    table_elements.append(Element(type="table", text=str(str_df)))


In [10]:
print(table_elements[4])
print(type(table_elements[0]))

type='table' text='Unnamed: 0,Unnamed: 1,Unnamed: 2,M463,Unnamed: 3\r\nPART NUMBER,,,,\r\n,KGCAE,SGCAE,,LGCAE YGCAE\r\nSystem Frequency (MHz),,,200,\r\nFlash (KB),,,256,\r\nSRAM (KB),,,128,\r\nLDROM (KB),,,8,\r\nXOM (regions),,,4,\r\nPDMA,,,16-ch,\r\nI/O,100,44,,33 33\r\nRTC (VBAT),,,√,\r\n32-bit Timer,,,4,\r\n16-bit EPWM,,,12,\r\n16-bit BPWM,,,12,\r\nUART,,,8,\r\nQSPI,,,2,\r\nSPI/I2S,,,4,\r\nI2C,,,5,\r\nCAN FD,,,2,\r\nPSIO,,,1,\r\nUSCI,,,1,\r\nSD Host,,,1,\r\nISO-7816-3,,,1,\r\nUSB High Speed OTG with PHY,,,√,\r\nLCD Parallel Data Bus,,,,\r\n,,,√,\r\n(External Bus Interface),,,,\r\nEQEI,,,2,\r\nECAP,,,2,\r\n,,,,M463/M467 SERIES TECHNICAL REFERENCE MANUAL\r\nKPI,,,6x8,\r\n12-bit ADC,16,16,,12 12\r\nAnalog Comparator,,,2,\r\nCrypto,,,AES-256,\r\nTRNG,,,√,\r\nOperating Temperature,,,-40°C ~ 105°C,\r\n,LQFP 128,LQFP 64,,LQFP 48 QFN 48\r\nPackage,,,,\r\n,(14x14mm),(7x7mm),,(7x7mm) (5x5mm)\r\n'
<class '__main__.Element'>


## 3. Check the table & text elements

In [11]:
print("text element: {}".format(len(text_elements)))
print("table elements: {}".format(len(table_elements)))

text element: 2779
table elements: 4004


In [42]:
#text_elements_test = text_elements[0:3]
#table_elements_test = table_elements[0:5]

## 4. Create summaries
- Need LLM to crete each element's summary

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI

In [13]:
# Prompt
prompt_text = """You are an assistant tasked with summarizing tables and text. \ 
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatOpenAI(temperature=0, model="gpt-4-0125-preview")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

C:\ProgramData\miniforge3\envs\LC_gradio\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


- Create tables summaries

In [14]:
from tqdm.notebook import tqdm
# Apply to tables
#tables = [i.text for i in table_elements]
#table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

# In this way to save temp lists
tables = [i.text for i in table_elements]
table_summaries = []
split = 50
for i in tqdm(range(0, len(tables), split)):
    if (i+split) <= len(tables):
        buf_list = summarize_chain.batch(tables[i:i+split], {"max_concurrency": 5})
    else: 
        buf_list = summarize_chain.batch(tables[i:len(texts)], {"max_concurrency": 5})
    table_summaries.append(buf_list)

In [15]:
import pickle
from itertools import chain

table_summaries = list(chain.from_iterable(table_summaries))
print(len(table_summaries))

with open("save_summary_tables_buf.pickle", "wb") as f:
    pickle.dump(table_summaries, f)

- Create texts summaries

In [22]:
# Apply to texts
#texts = [i.text for i in text_elements]
#text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

# In this way to save temp lists
texts = [i.text for i in text_elements]
text_summaries = []
split = 50
for i in tqdm(range(0, len(texts), split)):
    if (i+split) <= len(texts):
        buf_list = summarize_chain.batch(texts[i:i+split], {"max_concurrency": 5})
    else: 
        buf_list = summarize_chain.batch(texts[i:len(texts)], {"max_concurrency": 5})
    text_summaries.append(buf_list)

  0%|          | 0/56 [00:00<?, ?it/s]

In [32]:
text_summaries = list(chain.from_iterable(text_summaries))
print(len(text_summaries))

with open("save_summary_texts_buf.pickle", "wb") as f:
    pickle.dump(text_summaries, f)

2779


## 5. Save the text&table elements
- Because we want to load these elements & summaries again without wasting time & money.

In [33]:
import pickle
import uuid

PIK = DOC_SAVE_NAME + r".pickle"

# encode the ids with text/table source. the ids[] will be used latter
doc_ids = [str(uuid.uuid4()) for _ in texts]
texts_list = list(zip(doc_ids, texts))

table_ids = [str(uuid.uuid4()) for _ in tables]
table_list = list(zip(table_ids, tables))

SUM_list = [texts_list, table_list]

# This will be used
with open(PIK, "wb") as f:
    pickle.dump(SUM_list, f)

# The summaries is only for backup
summaries_list = [text_summaries, table_summaries]
with open(DOC_SAVE_NAME + r"_summary.pickle", "wb") as f:
    pickle.dump(summaries_list, f)

In [34]:
with open(PIK, "rb") as f:
    SUM_list = pickle.load(f)

In [35]:
print(len(SUM_list))
print(len(SUM_list[0]))
print(len(SUM_list[1]))
print(SUM_list[1][4])

2
2779
4004
('a13df53a-fcd1-47bd-8d3c-b620d2f2bb72', 'Unnamed: 0,Unnamed: 1,Unnamed: 2,M463,Unnamed: 3\r\nPART NUMBER,,,,\r\n,KGCAE,SGCAE,,LGCAE YGCAE\r\nSystem Frequency (MHz),,,200,\r\nFlash (KB),,,256,\r\nSRAM (KB),,,128,\r\nLDROM (KB),,,8,\r\nXOM (regions),,,4,\r\nPDMA,,,16-ch,\r\nI/O,100,44,,33 33\r\nRTC (VBAT),,,√,\r\n32-bit Timer,,,4,\r\n16-bit EPWM,,,12,\r\n16-bit BPWM,,,12,\r\nUART,,,8,\r\nQSPI,,,2,\r\nSPI/I2S,,,4,\r\nI2C,,,5,\r\nCAN FD,,,2,\r\nPSIO,,,1,\r\nUSCI,,,1,\r\nSD Host,,,1,\r\nISO-7816-3,,,1,\r\nUSB High Speed OTG with PHY,,,√,\r\nLCD Parallel Data Bus,,,,\r\n,,,√,\r\n(External Bus Interface),,,,\r\nEQEI,,,2,\r\nECAP,,,2,\r\n,,,,M463/M467 SERIES TECHNICAL REFERENCE MANUAL\r\nKPI,,,6x8,\r\n12-bit ADC,16,16,,12 12\r\nAnalog Comparator,,,2,\r\nCrypto,,,AES-256,\r\nTRNG,,,√,\r\nOperating Temperature,,,-40°C ~ 105°C,\r\n,LQFP 128,LQFP 64,,LQFP 48 QFN 48\r\nPackage,,,,\r\n,(14x14mm),(7x7mm),,(7x7mm) (5x5mm)\r\n')


In [36]:
texts_list = SUM_list[0]
table_list = SUM_list[1]

## 6. Add to vectorstore and save vectorstore
- vectorstore: save summary_texts
- InMemoryStore: save original texts & tables load from pickles

In [37]:
from langchain.storage import InMemoryStore
import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_core.documents import Document

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

embedding=OpenAIEmbeddings()
#vectorstore=FAISS.from_texts(Document(page_content='test', metadata={id_key: '1111-1111'}), embedding)
vectorstore=FAISS.from_texts(' ', embedding, ids='1')

C:\ProgramData\miniforge3\envs\LC_gradio\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [38]:
# The vectorstore to use to index the child chunks
#vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

# Add texts
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(texts_list)

# Add tables
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(table_list)

In [ ]:
# delete the initial vector
vectorstore.delete('1')
#print(vectorstore.index_to_docstore_id)

In [40]:
vectorstore.save_local(DOC_SAVE_NAME)